In [38]:
from sage.all import *
import random
import binascii
import time

# Step 1: Calculate N = n1*n2*...
def calculate_N(n_list):
    N = 1
    for n in n_list:
        N *= n
    return N

# Step 2: Calculate each element T[j] using the Chinese Remainder Theorem (CRT)
def calculate_T(N, e):
    
    T = [] 
    for i in range(e):
        CRT_array = [0]*e
        CRT_array[i] = 1
        T.append(crt(CRT_array, N))
    return T

# Step 3: Assign P = PolynomialRing(Zmod(N)) &  Calculate g[j] = (mx+c)^e - c
def create_polynomial_ring_and_calculate_g(N, pad, const, e, c):
    P.<x> = PolynomialRing(Zmod(N))
    g = []
    for i in range(e):
        g.append((pad[i]*x+const[i])^e - Integer(c[i]))
    return g


# Step 4: Assign g_sum = sum_of(T[j] * g[j])
def calculate_g_sum(g, T):
    
    for i in range(len(T)):
        g[i] = T[i]*g[i]

    # Perform the addition
    g_sum = sum(g)
    return g_sum

# Step 5: Check if g is a monic polynomial, if not transform it into a monic polynomial
def check_monic_polynomial(g_sum):
    if g_sum.leading_coefficient() != 1:
        g_sum = g_sum.monic()
    return g_sum

# Step 6: Find small roots of g and check if that is the message
def find_roots(g_sum):
    roots = g_sum.small_roots()
    return roots

# Converts a string to hex before converting it to int
def convert_to_int(message):
    return int(binascii.hexlify(message.encode()), 16)

# Convert the int to hex and remove the prefix 0x then convert it back to str
def convert_to_str(num):
    return binascii.unhexlify(hex(num)[2:]).decode()


mod_array, cipher_array, pad_array, const_array = [], [], [], []
initial_message = "passwordtodayisabc123"
print(f"The initial message is: {initial_message}")
m = convert_to_int(initial_message)
e = 3
pad_bound = 2^256 # Upperbound for the linear function values
prime_bound = 2^512

for i in range(e):
    pad = random.randint(0,pad_bound)
    constant = random.randint(0,pad_bound)
    pad_array.append(pad) 
    const_array.append(constant)
    p = random_prime(prime_bound,proof=false)
    q = random_prime(prime_bound,proof=false)
    n = p*q
    mod_array.append(n)
    cipher_array.append(pow(pad * m + constant,e,n))

if not(len(cipher_array) == len(mod_array) == len(pad_array) == len(const_array) == e):
    raise Exception("Number of Polynomials too small")

start_time = time.time()
N = calculate_N(mod_array)
T = calculate_T(mod_array, e) 
g = create_polynomial_ring_and_calculate_g(N, pad_array, const_array, e, cipher_array)
g_sum = calculate_g_sum(g, T)
g_sum = check_monic_polynomial(g_sum)
roots = find_roots(g_sum)
time_taken = time.time() - start_time

if len(roots) != 0:
    print(f"Found the message: {convert_to_str(roots[0])}")
    print(("in: %s seconds " % time_taken))
else:
    print("No roots found")

The initial message is: passwordtodayisabc123
Found the message: passwordtodayisabc123
in: 0.17877411842346191 seconds 
